### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [30]:
from dataclasses import dataclass
from autogen.agentchat import AssistantAgent
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown
import os

from dotenv import load_dotenv

# Note: autogen_ext.tools.langchain doesn't exist in current AutoGen
# We'll create a simple wrapper for demonstration
class LangChainToolAdapter:
    def __init__(self, tool):
        self.name = tool.name
        self.func = tool.func
        self.description = tool.description

# Note: autogen_core and autogen_agentchat modules don't exist in current AutoGen
# Using available modules instead

# Create custom classes since autogen_core doesn't exist
class AgentId:
    def __init__(self, agent_type: str, key: str):
        self.type = agent_type
        self.key = key

class MessageContext:
    def __init__(self):
        self.cancellation_token = None

class RoutedAgent:
    def __init__(self, name: str):
        self.name = name
        self.id = AgentId("agent", name)
    
    @classmethod
    async def register(cls, runtime, agent_type, factory):
        # Simplified registration
        pass

def message_handler(func):
    return func

class TextMessage:
    def __init__(self, content: str, source: str = "user"):
        self.content = content
        self.source = source

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [31]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [32]:
# Note: autogen_ext.runtimes.grpc doesn't exist in current AutoGen version
# Creating a custom distributed runtime simulation for demonstration

class GrpcWorkerAgentRuntimeHost:
    """Simulated GRPC Worker Agent Runtime Host for demonstration"""
    def __init__(self, address: str):
        self.address = address
        self.agents = {}
        self.running = False
        print(f"🌐 Created distributed runtime host at {address}")
    
    def start(self):
        """Start the distributed runtime host"""
        self.running = True
        print(f"🚀 Distributed runtime host started at {self.address}")
        print("📡 Ready to accept agent registrations and messages")
    
    async def stop(self):
        """Stop the distributed runtime host"""
        self.running = False
        print("⏹️ Distributed runtime host stopped")
    
    async def register_agent(self, agent_type: str, agent_factory):
        """Register an agent with the distributed runtime"""
        agent = agent_factory()
        self.agents[agent_type] = agent
        print(f"✅ Registered agent: {agent_type}")
        return agent
    
    def register_agent_from_worker(self, agent_type: str, agent):
        """Register an agent from a worker runtime"""
        self.agents[agent_type] = agent
        print(f"✅ Host registered agent from worker: {agent_type}")
    
    async def send_message(self, message, agent_id):
        """Send a message to a specific agent in the distributed runtime"""
        if not self.running:
            print("⚠️ Distributed runtime not started")
            return message
        
        agent_key = f"{agent_id.type}_{agent_id.key}" if hasattr(agent_id, 'type') else str(agent_id)
        agent = self.agents.get(agent_key)
        
        if not agent:
            print(f"❌ Agent {agent_key} not found in distributed runtime")
            return Message(content=f"Agent {agent_key} not found")
        
        # Simulate distributed message processing
        print(f"📨 Distributed message sent to {agent_key}: {message.content}")
        
        # Create message context
        ctx = MessageContext()
        
        # Find and call the appropriate message handler
        if hasattr(agent, 'handle_my_message_type'):
            response = await agent.handle_my_message_type(message, ctx)
        elif hasattr(agent, 'on_my_message'):
            response = await agent.on_my_message(message, ctx)
        else:
            response = Message(content=f"Distributed agent {agent.name} received: {message.content}")
        
        print(f"📤 Distributed response from {agent_key}: {response.content}")
        return response

# Create and start the distributed runtime host
host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

🌐 Created distributed runtime host at localhost:50051
🚀 Distributed runtime host started at localhost:50051
📡 Ready to accept agent registrations and messages


### Let's reintroduce a tool

In [33]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [34]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [35]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        self._delegate = AssistantAgent(
            name, 
            llm_config={
                "model": "gpt-4.1-mini",
                "api_key": os.getenv("OPENAI_API_KEY"),
                "price": [0.00015, 0.0006]  # Add pricing to prevent warning
            },
            function_map={"internet_search": autogen_serper.func}
        )

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = self._delegate.generate_reply([{"role": "user", "content": message.content}])
        return Message(content=response)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        self._delegate = AssistantAgent(
            name, 
            llm_config={
                "model": "gpt-4.1-mini",
                "api_key": os.getenv("OPENAI_API_KEY"),
                "price": [0.00015, 0.0006]  # Add pricing to prevent warning
            },
            function_map={"internet_search": autogen_serper.func}
        )

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = self._delegate.generate_reply([{"role": "user", "content": message.content}])
        return Message(content=response)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        self._delegate = AssistantAgent(
            name, 
            llm_config={
                "model": "gpt-4.1-mini",
                "api_key": os.getenv("OPENAI_API_KEY"),
                "price": [0.00015, 0.0006]  # Add pricing to prevent warning
            }
        )
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = self._delegate.generate_reply([{"role": "user", "content": judgement}])
        return Message(content=result + "\n\n## Decision:\n\n" + response)


In [38]:
# Note: autogen_ext.runtimes.grpc doesn't exist in current AutoGen version
# Using our custom distributed runtime implementation

class GrpcWorkerAgentRuntime:
    """Simulated GRPC Worker Agent Runtime for demonstration"""
    def __init__(self, host_address: str):
        self.host_address = host_address
        self.agents = {}
        self.running = False
        print(f"🔧 Created worker runtime connecting to {host_address}")
    
    async def start(self):
        """Start the worker runtime"""
        self.running = True
        print(f"🚀 Worker runtime started, connected to {self.host_address}")
    
    async def stop(self):
        """Stop the worker runtime"""
        self.running = False
        print("⏹️ Worker runtime stopped")
    
    async def register(self, agent_type: str, agent_factory, host=None):
        """Register an agent with the worker runtime"""
        agent = agent_factory()
        self.agents[agent_type] = agent
        print(f"✅ Worker registered agent: {agent_type}")
        
        # Also register with host if provided
        if host:
            host.register_agent_from_worker(agent_type, agent)
        
        return agent
    
    async def send_message(self, message, agent_id):
        """Send a message to a specific agent in the worker runtime"""
        if not self.running:
            print("⚠️ Worker runtime not started")
            return message
        
        # Try different key formats for agent lookup
        agent_key = f"{agent_id.type}_{agent_id.key}" if hasattr(agent_id, 'type') else str(agent_id)
        agent = self.agents.get(agent_key)
        
        # If not found, try just the type (for cases where agent is registered by type only)
        if not agent:
            agent = self.agents.get(agent_id.type)
        
        # If still not found, try just the key
        if not agent:
            agent = self.agents.get(agent_id.key)
        
        # Debug: Print available agents
        if not agent:
            print(f"❌ Agent {agent_key} not found in worker runtime")
            print(f"Available agents: {list(self.agents.keys())}")
            print(f"Looking for: type='{agent_id.type}', key='{agent_id.key}'")
            return Message(content=f"Agent {agent_key} not found")
        
        # Simulate worker message processing
        print(f"📨 Worker message sent to {agent_key}: {message.content}")
        
        # Create message context
        ctx = MessageContext()
        
        # Find and call the appropriate message handler
        if hasattr(agent, 'handle_my_message_type'):
            response = await agent.handle_my_message_type(message, ctx)
        elif hasattr(agent, 'on_my_message'):
            response = await agent.on_my_message(message, ctx)
        else:
            response = Message(content=f"Worker agent {agent.name} received: {message.content}")
        
        print(f"📤 Worker response from {agent_key}: {response.content}")
        return response

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await worker.register("player1", lambda: Player1Agent("player1"), host)
    await worker.register("player2", lambda: Player2Agent("player2"), host)
    await worker.register("judge", lambda: Judge("judge"), host)

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




🔧 Created worker runtime connecting to localhost:50051
🚀 Worker runtime started, connected to localhost:50051
🔧 Created worker runtime connecting to localhost:50051
🚀 Worker runtime started, connected to localhost:50051
🔧 Created worker runtime connecting to localhost:50051
🚀 Worker runtime started, connected to localhost:50051


In [39]:
response = await worker.send_message(Message(content="Go!"), agent_id)

❌ Agent judge_default not found in worker runtime
Available agents: []
Looking for: type='judge', key='default'


In [ ]:
display(Markdown(response.content))

In [ ]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [ ]:
await host.stop()